In [1]:
import os
import pandas as pd
import numpy as np
from skimage.io import imread

from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense # the two types of neural network layer we will be using

# Set path
path = os.path.expanduser('~') + '/proga/kaggle_julia/JuliaKaggle-master'

/Users/romankucev/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1076: UserWarning: Bad val "TkAgg or echo backend : Agg" on line #1
	"backend : TkAgg or echo backend : Agg
"
	in file "/Users/romankucev/.matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string "tkagg or echo backend : agg": valid strings are [u'pgf', u'cairo', u'MacOSX', u'CocoaAgg', u'gdk', u'ps', u'GTKAgg', u'nbAgg', u'GTK', u'Qt5Agg', u'template', u'emf', u'GTK3Cairo', u'GTK3Agg', u'WX', u'Qt4Agg', u'TkAgg', u'agg', u'svg', u'GTKCairo', u'WXAgg', u'WebAgg', u'pdf']
  (val, error_details, msg))
Using Theano backend.


In [2]:
# Define functions
def read_data(typeData, labelsInfo, imageSize, path):
    x = np.zeros((labelsInfo.shape[0], imageSize))
    
    for (index, idImage) in enumerate(labelsInfo['ID']):
        # use specially created 32 x 32 images
        nameFile = '{0}/{1}Resized32/{2}.Bmp'.format(path, 
                    typeData, idImage)
        img = imread(nameFile, as_grey = True)
        
        x[index, :] = np.reshape(img, (1, imageSize))
        
    return x

In [3]:
height, width, depth = 32, 32, 1 
labelsInfoTrain = pd.read_csv('{0}/trainLabels.csv'.format(path))
labelsInfoTest = pd.read_csv('{0}/sampleSubmission.csv'.format(path))

# Load dataset
xTrain = read_data('train', labelsInfoTrain, height * width, path)
xTest = read_data('test', labelsInfoTest, height * width, path)

yTrain = map(ord, labelsInfoTrain['Class'])
yTrain = np.array(yTrain)

In [4]:
# Preprocess (Normalize) data
xTrain /= xTrain.std(axis = None)
xTrain -= xTrain.mean()

xTest /= xTest.std(axis = None)
xTest -= xTest.mean()

In [5]:
num_classes = 62
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 20 # we iterate twenty times over the entire training set
hidden_size = 512 # there will be 512 neurons in both hidden layers

In [6]:
inp = Input(shape=(height * width,)) # Our input is a 1D vector of size 1024
hidden_1 = Dense(hidden_size, activation='relu')(inp) # First hidden ReLU layer
hidden_2 = Dense(hidden_size, activation='relu')(hidden_1) # Second hidden ReLU layer
out = Dense(num_classes, activation='softmax')(hidden_2) # Output softmax layer

model = Model(input=inp, output=out) 

In [7]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [8]:
s = list(set(yTrain)) #set of 62 classes
Y = np.zeros((len(yTrain),62) ,dtype= np.int32 )
for ii,i in enumerate(yTrain):
    Y[ii][s.index(i)] = 1

In [9]:
model.fit(xTrain, Y, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Train on 5654 samples, validate on 629 samples
Epoch 1/20
5654/5654 [==============================] - 1s - loss: 3.6503 - acc: 0.1597 - val_loss: 3.2123 - val_acc: 0.2258
Epoch 2/20
5654/5654 [==============================] - 1s - loss: 2.7896 - acc: 0.3449 - val_loss: 2.5786 - val_acc: 0.3720
Epoch 3/20
5654/5654 [==============================] - 1s - loss: 2.2061 - acc: 0.4616 - val_loss: 2.1705 - val_acc: 0.4706
Epoch 4/20
5654/5654 [==============================] - 1s - loss: 1.8353 - acc: 0.5405 - val_loss: 1.9945 - val_acc: 0.5056
Epoch 5/20
5654/5654 [==============================] - 1s - loss: 1.5829 - acc: 0.5996 - val_loss: 1.8467 - val_acc: 0.5151
Epoch 6/20
5654/5654 [==============================] - 1s - loss: 1.3667 - acc: 0.6459 - val_loss: 1.7669 - val_acc: 0.5374
Epoch 7/20
5654/5654 [==============================] - 1s - loss: 1.2164 - acc: 0.6797 - val_loss: 1.6651 - val_acc: 0.5533
Epoch 8/20
5654/5654 [==============================] - 1s - loss: 1.0794 - ac

In [10]:
pred = model.predict(xTest)
ans = []
for i in pred:
    ans.append(s[i.argmax()])

In [11]:
# Convert for submission
yTest = map(chr, ans)
labelsInfoTest['Class'] = yTest
labelsInfoTest.to_csv('{0}/submission.csv'.format(path), index = False)